

---



# План анализа данных

  1. Загрузить данные для обучения;
  2. Обработать данные перед обучением модели;
  3. Обучить и провалидировать модель;
  4. Попробовать улучшить модель с помощью подбора параметров.



---

# Установка библиотек

In [1]:
# Grab Jaime's excellent condacolab package: https://github.com/jaimergp/condacolab
# Note: you should probably read the README file at that repo.
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:35
🔁 Restarting kernel...


In [1]:
!mamba install -c conda-forge rdkit chembl_structure_pipeline


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.8.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit', 'chembl_structure_pipeline']

conda-forge/linux-64     Using cache
conda-forge/noarch     

In [2]:
!pip install git+https://github.com/bp-kelley/descriptastorus

  Cloning https://github.com/bp-kelley/descriptastorus to /tmp/pip-req-build-5n9vojkn
  Running command git clone -q https://github.com/bp-kelley/descriptastorus /tmp/pip-req-build-5n9vojkn
     |████████████████████████████████| 801 kB 5.1 MB/s 
  Created wheel for descriptastorus: filename=descriptastorus-2.3.0.2-py3-none-any.whl size=60174 sha256=17439c7cca0a7ae85755649962330376b7fe5c51dc7b42fef6a066a668d83686
  Stored in directory: /tmp/pip-ephem-wheel-cache-436z0ec7/wheels/f9/c3/4f/e7d01f4f2f1a89aef8f0ef088beb4a94976324f3ee21410b10
Successfully built descriptastorus


# Импорт библиотек

In [3]:
import pandas as pd
import os

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from typing import Dict, List, Optional
from pathlib import Path

from multiprocessing.pool import Pool

from rdkit.Chem.MolStandardize.tautomer import TautomerCanonicalizer



# Загрузка данных

In [39]:
!wget https://www.dropbox.com/s/5b05tivi01a43np/delaney-processed.csv
!wget https://www.dropbox.com/s/dsrl00tj4zjmqbz/Lipophilicity.csv

--2021-05-09 09:56:11--  https://www.dropbox.com/s/5b05tivi01a43np/delaney-processed.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5b05tivi01a43np/delaney-processed.csv [following]
--2021-05-09 09:56:11--  https://www.dropbox.com/s/raw/5b05tivi01a43np/delaney-processed.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc55f45ff744e818270390fc41f5.dl.dropboxusercontent.com/cd/0/inline/BOIPMih-L0D9coG_LXXv1nX-5t2GXL52BHir1sRCNg7Jp7-J6Geb1yqv8gWdG0qMxXHzk4Duw9qmLxLaJ6ooWd7pJ8nXpnw1exrorvdah_9OydMhvu5REuIQ-FJZGTWnotHORICzvCu_oZ_1FPbkI7K_/file# [following]
--2021-05-09 09:56:11--  https://uc55f45ff744e818270390fc41f5.dl.dropboxusercontent.com/cd/0/inline/BOIPMih-L0D9coG_LXXv1nX-5t2GXL52BHir1sRCNg7Jp7-J6Geb1yqv8gWdG0q

In [6]:
class DatasetsHolder:
    @staticmethod
    def read_datasets(inp_folder_path):
        # return pandas DataFrame
        df = pd.read_csv(inp_folder_path)
        return df


class StandardizeDatasets:
    @staticmethod
    def standardize_smiles(smi: str) -> Optional[str]:
        "crete typical standartization of one smiles"
        mol = Chem.MolFromSmiles(smi)
        smi1 = Chem.MolToSmiles(mol)
        return smi1

    def standardize(self, inp_path: Path, out_path: Path):
        "apply standartization to all smiles"
        dt = DatasetsHolder()
        df = dt.read_datasets(inp_path)
        sd = StandardizeDatasets()
        df['standardized_smiles'] = df['smiles'].apply(sd.standardize_smiles)
        df.to_csv(out_path)


class StandardizeTautomers(StandardizeDatasets):
    @staticmethod
    def standardize_smiles(smi: str) -> Optional[str]:
        "apply TautomerCanonicalizer() to standartization"
        tc = TautomerCanonicalizer()
        mol = Chem.MolFromSmiles(smi)
        mol1 = tc.canonicalize(mol)
        smi1 = Chem.MolToSmiles(mol)
        return smi1

In [41]:
stantaut = StandardizeTautomers()
stantaut.standardize('delaney-processed.csv', 'out-delaney-processed.csv')
stantaut.standardize('Lipophilicity.csv', 'out-Lipophilicity.csv')

In [43]:
SMILES_COLUMN = 'standardized_smiles' # название предобработанных данных
VALUE_COLUMN = 'expt'

DATASET_INPUT_PATH = 'out-delaney-processed.csv' #название предоработанного файла
DATASET_INPUT_PATH_lip = 'out-Lipophilicity.csv' #название предоработанного файла

# Выделение признаков

In [44]:
data_reader = DatasetsHolder()#класс, читающий данные
data = data_reader.read_datasets(DATASET_INPUT_PATH)# метод
data_lip = data_reader.read_datasets(DATASET_INPUT_PATH_lip)# метод

In [48]:
data.head()
data_lip.head()

,Unnamed: 0,CMPD_CHEMBLID,exp,smiles,standardized_smiles
0,0,CHEMBL596271,3.54,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,Cn1c(CN2CCN(c3ccc(Cl)cc3)CC2)nc2ccccc21
1,1,CHEMBL1951080,-1.18,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,COc1cc(OC)c(S(=O)(=O)N2c3ccccc3CCC2C)cc1NC(=O)...
2,2,CHEMBL1771,3.69,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,COC(=O)[C@H](c1ccccc1Cl)N1CCc2sccc2C1
3,3,CHEMBL234951,3.37,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,O=C(NC1Cc2ccccc2N(C[C@@H](O)CO)C1=O)c1cc2cc(Cl...
4,4,CHEMBL565079,3.10,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,Cc1cccc(C[C@H](NC(=O)c2cc(C(C)(C)C)nn2C)C(=O)N...


In [49]:
from descriptastorus.descriptors import rdDescriptors
from rdkit import Chem
import logging


generator = rdDescriptors.RDKit2D()


def rdkit_2d_features(smiles: str):
    # Достать свойства молекулы при помощи https://github.com/bp-kelley/descriptastorus
    features = generator.process(smiles)
    if not features[0]:
        print(f'{smiles} not processed')
        return None
    return features[1:]
    

In [51]:
def create_feature_dataframe(df, target):
    features_names = [gen[0] for gen in generator.columns]
    dicts = []
    for i in range(len(df)):
        x, y = df.iloc[i]['smiles'], df.iloc[i][target]
        features = rdkit_2d_features(x)
        dicts.append(dict(zip(features_names, features)))
        dicts[-1]['target'] = y
    df1 = pd.DataFrame(dicts)
    return df1



In [52]:
data_feats = create_feature_dataframe(data, 'measured log solubility in mols per litre')
data_feats_lip = create_feature_dataframe(data_lip, 'exp')

# Деление данных на тренировочну и тестовую выборки

In [18]:
data_feats.head()

,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,ExactMolWt,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,FractionCSP3,HallKierAlpha,HeavyAtomCount,HeavyAtomMolWt,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,MaxAbsEStateIndex,MaxAbsPartialCharge,...,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,qed,target
0,1.654937,759.662938,23.413485,16.862520,16.862520,15.277295,9.998816,9.998816,7.601218,7.601218,5.431494,5.431494,3.506930,3.506930,80.729515,41.007583,0.0,0.00000,5.563451,0.000000,0.000000,30.331835,6.069221,0.000000,18.947452,457.158411,0.812500,1.375000,1.968750,0.650000,-1.73,32,430.216,1.212120e+07,24.903474,10.926356,5.251706,182.935327,10.253329,0.393567,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.217518,-0.77
1,2.148162,459.484175,10.673362,8.357948,8.357948,7.270857,4.676643,4.676643,3.210611,3.210611,2.135103,2.135103,1.340444,1.340444,0.000000,4.794537,0.0,5.90718,11.323699,5.687386,6.263163,12.990104,30.331835,5.316789,4.417151,201.078979,1.200000,1.933333,2.533333,0.083333,-2.03,15,190.137,4.231896e+03,9.522160,4.002882,2.070849,87.724095,11.724911,0.468799,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.811283,-3.30
2,3.625760,171.311799,8.690234,7.554513,7.554513,5.163902,3.908188,3.908188,2.969252,2.969252,1.443820,1.443820,0.788002,0.788002,0.000000,4.794537,0.0,0.00000,0.000000,24.700908,5.573105,6.076020,6.923737,19.923495,0.000000,152.120115,1.272727,1.909091,2.363636,0.500000,-0.85,11,136.109,2.036951e+02,10.150000,5.899351,7.042356,68.806046,10.020498,0.298566,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.343706,-2.06
3,2.041379,1071.547817,14.518297,12.082904,12.082904,10.915816,7.636751,7.636751,5.829201,5.829201,4.648219,4.648219,3.586716,3.586716,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,43.089794,0.000000,0.000000,84.929139,0.000000,278.109550,0.272727,0.636364,1.136364,0.000000,-2.86,22,264.242,2.961396e+05,11.762233,4.315741,1.523286,128.158061,2.270278,0.061629,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.291526,-7.87
4,3.125000,60.124818,3.535534,2.717649,3.534146,2.500000,1.471405,2.414214,0.793148,1.609645,0.425381,1.053920,0.226805,0.680414,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,11.336786,22.892860,0.000000,0.000000,84.003371,1.000000,1.600000,1.800000,0.000000,-0.30,5,80.111,2.288644e+01,2.912766,1.221050,0.484065,35.071766,2.041667,0.152454,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.448927,-1.33


In [53]:
train_data, test_data = train_test_split(data_feats)
X = train_data.drop('target', axis=1)
y = train_data['target']

train_data_lip, test_data_lip = train_test_split(data_feats_lip)
X_lip = train_data_lip.drop('target', axis=1)
y_lip = train_data_lip['target']

# Создание модели

In [54]:
from xgboost import XGBRegressor

In [55]:
regr = XGBRegressor()

# Обучение и валидация

![alt text](https://drive.google.com/uc?id=1Ilkmp248M0kKA3wFJQNQcNEY9OFsVoWz)

Стандартно для правильной валидации модели используют отложенную выборку. То есть мы разбиваем наши данные на **тренировочную** выборку, **тестовую** выборку и **отложенную** выборку. Соответственно, обучаем модель на тренировочной, в ходе обучения проверяем результат на тестовой выборке, а в конце обучения, чтобы оценить качество модели, ошибку считаем на отложенной выборке.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1wgVvskPBQJgiRwpsHUmUOS-MhfBatsWy" 
alt="IMAGE ALT TEXT HERE" width="480" border="0" /></a>

*Замечание:* тестовая и отложенная выборка могут совпадать. Главное - на этой части данных модель не обучается!


Однако, при таком подходе в обучении модели участвует только тренировочная выборка. Тестовую и отложенную мы используем только для проверки. Если у нас мало данных - это непозволительная роскошь. 

Другой популярный подход это **кросс-валидация** или скользящий контроль. Суть метода заключается в том, что мы делаем не одно разбиение датесета, а несколько разбиений таким образом, чтобы все данные использовались и в обучении и для проверки. Такие разбиения называются **фолдами**. 

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=14fZpuBDsTMqv1XtLJvcKMNNa1vlr_ZG6" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>


Преимущества такого подхода:
* используем все данные для обучения;
* можем оценить устойчивость модели. Если ошибки полученные на разных фолдах сильно отличаются, что модель неустойчива.

Недостаток метода в том, что нам нужно обучать не одну модель, а несколько (столько, сколько мы выбрали фолдов).

На практике часто выбирают 5 фолдов.

In [56]:
from sklearn.model_selection import KFold, cross_val_score

In [57]:
# создать делитель для кросс-валидации
kf = KFold(n_splits=5)

Функция `cross_val_score` воспроизводит разбиение, обучение и тестирование в соответствие с типом и параметрами передаваемого в нее валидатора. 

В нее передаем оцениваемую модель, таблицу входных данных, выходную переменную, способ разделения данных (фолды) и метрику, которую мы хотим оценить. В данном случае мы хотим оценить **r2_score**.

На выходе получим значения метрик. Так как мы передали в `KFold` с параметром **n_splits=5**, то и значений мы получим **5**.

In [31]:
# посчитать rmse метрику для модели
import numpy as np
vals = cross_val_score(regr, X, y, cv=kf, scoring='neg_root_mean_squared_error')
print(vals)
print(np.mean(vals), np.std(vals))

[09:43:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:43:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:43:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:43:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:43:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[-0.69889014 -0.6128675  -0.76580634 -0.64499144 -0.59660251]
-0.6638315843431948 0.06180613124942473


# Поиск по сетке



<a href="https://drive.google.com/uc?id=1Goc0VR5I--q9rYj-vYlmddanKP3-3sLJ
" target="_blank"><img src="https://drive.google.com/uc?id=1Goc0VR5I--q9rYj-vYlmddanKP3-3sLJ" 
alt="IMAGE ALT TEXT HERE" width="480" border="0" /></a>


## Поиск по сетке. 

Вместо того, чтобы перебирать параметры руками, можно использовать метод **поиска по сетке** (Grid Search). В процессе поиска по сетке мы указываем варианты каждого из параметров, которые хотим перебрать, а функция смотрит на все их возможные варианты и выдает лучший набор в зависимости от выбранной метрики. Например, на картинке ниже перебираются параметры "регуляризация" и "скорость обучения".

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1FhZpRMWuzCXQs1DDdTn11hjmH3MS6C6j" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

Поиск по сетке реализован в **sklearn**, импортируем его:

In [58]:
from sklearn.model_selection import GridSearchCV


In [59]:
# сделайте поиск по сетке
clf = GridSearchCV(regr, scoring='neg_root_mean_squared_error', param_grid = {
    "max_depth": [3,5,7,9], 
    "n_estimators": [50,100,150,200],
})
clf.fit(X, y)
res = clf.get_params()

[10:04:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:04:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:04:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:04:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:04:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:04:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:04:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:04:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

KeyboardInterrupt: ignored

In [62]:
clf = GridSearchCV(regr, scoring='r2', param_grid = {
    "max_depth": [3,5,7,9], 
    "n_estimators": [50,100,150,200],
})
clf.fit(X_lip, y_lip)
res = clf.get_params()

[10:12:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:12:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [63]:
res = clf.best_params_
print(clf.best_score_)
print(res)

0.6800303213095751
{'max_depth': 5, 'n_estimators': 200}
